In [ ]:
import os
import pydicom
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

gt_folder = '/Users/ryotaroakagawa/pix2pix3D-CT/dicom/dataset01_test'
# gene_folder = '/Users/ryotaroakagawa/pix2pix3D-CT/result/dataset01_test'
gene_folder = '/Users/ryotaroakagawa/pix2pix3D-CT/dicom/dataset01_test'

total_psnr = 0
total_folders = 0

case_folders = [folder for folder in sorted(os.listdir(gt_folder)) if 'case' in folder]

for case_folder in case_folders:
    gt_folder_path = os.path.join(gt_folder, case_folder, 'CT2')
    gene_folder_path = os.path.join(gene_folder, case_folder, 'CT1')
    # gene_folder_path = os.path.join(gene_folder, case_folder, 'dicom')
    
    gt_files = sorted(os.listdir(gt_folder_path))
    gene_files = sorted(os.listdir(gene_folder_path))

    case_psnr = 0

    for file_path in gt_files:
        gt_file_path = os.path.join(gt_folder_path, file_path)
        gene_file_path = os.path.join(gene_folder_path, file_path)

        gt_dicom = pydicom.dcmread(gt_file_path)
        gene_dicom = pydicom.dcmread(gene_file_path)

        # gt_array = gt_dicom.pixel_array * gt_dicom.RescaleSlope + gt_dicom.RescaleIntercept
        # gene_array = gene_dicom.pixel_array * gene_dicom.RescaleSlope + gene_dicom.RescaleIntercept
        gt_array = gt_dicom.pixel_array.astype(float)
        gene_array = gene_dicom.pixel_array.astype(float)

        # 画素値の範囲を手動で指定（16ビットの場合）
        data_range = 2 ** gt_dicom.BitsStored - 1

        psnr = peak_signal_noise_ratio(gt_array.flatten(), gene_array.flatten(), data_range=data_range)
        
        case_psnr += psnr

    avg_case_psnr = case_psnr / len(gt_files)
    total_psnr += avg_case_psnr
    total_folders += 1  
    print(f"Case {case_folder}: PSNR = {avg_case_psnr}")

print('Average PSNR:', total_psnr / total_folders)
